In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE111974"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE111974"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE111974.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE111974.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE111974.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Endometrial Tissue RNA expression in Recurrent Implantation Failure vs. Conrol"
!Series_summary	"We aimed to identify altered biological processes in the endometrium that may be potential markers of receptive endometrium. RNA expression profiling of the endometrium during the window of implantation was performed in patients with Recurrent Implantation Failure (RIF) versus fertile controls."
!Series_overall_design	"24 patients with RIF treated at the IVF clinic and 24 fertile control patients recruited from the gynecology clinic of Istanbul University School of Medicine during 2014-2015 were involved in this prospective cohort study. RIF was determined as failure of pregnancy in ≥ 3 consecutive IVF cycles with ≥1 transfer(s) of good quality embryo in each cycle. Exclusion criteria for this group were active pelvic infections, undiagnosed vaginal bleeding, uterine anomalies, endometriosis, karyotype anomalies in one or both partners. Fertile control

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Set gene_expression availability based on data review
# This dataset mentions RNA expression profiling, which indicates gene expression data is available
is_gene_available = True

# Check clinical data availability (trait, age, gender)
# For trait: The sample characteristics don't show RIF vs Control information
# And the extraction attempt resulted in all NaN values
trait_row = None  # No explicit trait information available in the expected format

# For age: Not mentioned in sample characteristics
age_row = None

# For gender: All subjects appear to be female by study design (IVF study)
gender_row = None

# Define conversion functions (needed for the interface but won't be used with None values)
def convert_trait(trait_value):
    """
    Convert trait values to binary (0 for control, 1 for RIF)
    """
    if trait_value is None:
        return None
    
    # Extract value after colon if present
    if ':' in trait_value:
        trait_value = trait_value.split(':', 1)[1].strip()
    
    trait_value = trait_value.lower()
    if 'rif' in trait_value or 'recurrent implantation failure' in trait_value:
        return 1
    elif 'control' in trait_value or 'fertile' in trait_value:
        return 0
    else:
        return None

def convert_age(age_value):
    """
    Convert age values to continuous
    """
    if age_value is None:
        return None
    
    # Extract value after colon if present
    if ':' in age_value:
        age_value = age_value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float, handling various formats
        age_value = age_value.replace('years', '').replace('year', '').strip()
        return float(age_value)
    except (ValueError, AttributeError):
        return None

def convert_gender(gender_value):
    """
    Convert gender values to binary (0 for female, 1 for male)
    """
    if gender_value is None:
        return None
    
    # Extract value after colon if present
    if ':' in gender_value:
        gender_value = gender_value.split(':', 1)[1].strip().lower()
    
    if 'female' in gender_value or 'f' == gender_value:
        return 0
    elif 'male' in gender_value or 'm' == gender_value:
        return 1
    else:
        return None

# Check trait availability based on trait_row
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Skip clinical feature extraction as trait_row is None


A new JSON file was created at: ../../output/preprocess/Endometriosis/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 56
Header line: "ID_REF"	"GSM3045867"	"GSM3045868"	"GSM3045869"	"GSM3045870"	"GSM3045871"	"GSM3045872"	"GSM3045873"	"GSM3045874"	"GSM3045875"	"GSM3045876"	"GSM3045877"	"GSM3045878"	"GSM3045879"	"GSM3045880"	"GSM3045881"	"GSM3045882"	"GSM3045883"	"GSM3045884"	"GSM3045885"	"GSM3045886"	"GSM3045887"	"GSM3045888"	"GSM3045889"	"GSM3045890"	"GSM3045891"	"GSM3045892"	"GSM3045893"	"GSM3045894"	"GSM3045895"	"GSM3045896"	"GSM3045897"	"GSM3045898"	"GSM3045899"	"GSM3045900"	"GSM3045901"	"GSM3045902"	"GSM3045903"	"GSM3045904"	"GSM3045905"	"GSM3045906"	"GSM3045907"	"GSM3045908"	"GSM3045909"	"GSM3045910"	"GSM3045911"	"GSM3045912"	"GSM3045913"	"GSM3045914"
First data line: "A_19_P00315452"	8.2941	9.4957	9.13	8.1259	8.2462	9.04	7.7973	8.5905	9.2121	9.0986	9.6616	8.5906	8.1014	9.2161	9.364	8.2461	8.4084	9.6027	7.2648	7.9788	8.4856	8.7482	9.1229	9.1373	8.5388	7.8161	7.3634	7.976	8.1333	7.6221	6.5153	9.2491	7.7401	7.9426	8.2897	8.1575	7.8499	7.3065	7.7341	8.6831	8.2265	8.6232	5.5

Index(['A_19_P00315452', 'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492',
       'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518',
       'A_19_P00315519', 'A_19_P00315524', 'A_19_P00315528', 'A_19_P00315529',
       'A_19_P00315538', 'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315550',
       'A_19_P00315551', 'A_19_P00315554', 'A_19_P00315581', 'A_19_P00315583'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# The identifiers like "A_19_P00315452" appear to be probe IDs from a microarray platform
# These are not standard human gene symbols and will need to be mapped to proper gene symbols

# Based on my biomedical knowledge, these "A_19_P" identifiers are Agilent microarray probe IDs
# They need to be mapped to standard gene symbols for proper interpretation and analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene identifier mapping
# From the gene annotation preview, we can see:
# - 'ID' column contains identifiers like A_23_P117082, which matches the indices in gene_data
# - 'GENE_SYMBOL' column contains gene symbols like HEBP1, which is what we need

# 2. Extract the ID and GENE_SYMBOL columns to create mapping dataframe
prob_col = 'ID'  # The column containing probe IDs
gene_col = 'GENE_SYMBOL'  # The column containing gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few rows of the gene expression data after mapping
print("Gene expression data preview after mapping:")
print(gene_data.index[:20])  # Show the first 20 gene symbols
print(f"Shape of gene expression data: {gene_data.shape}")


Gene expression data preview after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT',
       'A4GNT', 'AA06', 'AAA1', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACL3', 'AADACL4', 'AADAT', 'AAGAB'],
      dtype='object', name='Gene')
Shape of gene expression data: (20353, 48)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row is None (as determined in step 2), we use is_final=False for validation
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False  # No trait data available
)

# Create empty clinical features dataframe since trait_row is None
clinical_features_df = pd.DataFrame()
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data (empty) saved to {out_clinical_data_file}")

# No further processing needed as the dataset is unusable due to lack of trait data
print("Data was determined to be unusable due to missing trait information")

Normalized gene data saved to ../../output/preprocess/Endometriosis/gene_data/GSE111974.csv
Clinical data (empty) saved to ../../output/preprocess/Endometriosis/clinical_data/GSE111974.csv
Data was determined to be unusable due to missing trait information
